<h1>D-Wave Hybrid solver CQM sudoku solver<h1>

<h2>July 2022 by Emilio Pomares (epomares@swaap.us)<h2>

In [1]:
# Imports

from sudoku import Sudoku as s
import numpy as np
from dimod import SimulatedAnnealingSampler
from dimod import ConstrainedQuadraticModel
from dwave.system.composites import EmbeddingComposite
from dimod import Binary

from dwave.system import LeapHybridCQMSampler
from dwave.system import DWaveSampler

# We need a sampler that can work with CQM models:

sampler = LeapHybridCQMSampler(token='Your API Key here')

In [2]:
# Some helpers functions...

def get_board_value(i, j, output):
    '''
    Utility to reconstruct one-hot encoded number from solution
    '''
    for k in range(1,maxRange+1):
        if(output[f'SEL_{i}_{j}_{k}']) > 0.0:
            return k
    return None

def draw_row_separator():
    '''
    Draws a board row separator as ASCII characters
    '''
    tempstr = ""
    for w in range(0,size):
        tempstr += "+"
        for k in range(0,2*size+1):
            tempstr += "-"
    tempstr += "+"
    print(tempstr)

def draw_board(output):
    '''
    Draws the sudoku board as ASCII characters
    '''
    for i in range(1,maxRange+1):
        if (i-1)%size == 0:
            draw_row_separator()
        tempstr = ""
        for j in range(1,maxRange+1):
            if (j-1)%size == 0:
                tempstr += "| "
            tempstr += (str(get_board_value(i,j,output)) + " ")
        tempstr += "|"
        print(tempstr)
    tempstr = ""
    draw_row_separator()

In [3]:
# Define sizes

size = 3
maxRange = size*size

In [4]:
# Generate a puzzle and visualize it

puzzle = s(size).difficulty(0.8)
puzzle.show()

+-------+-------+-------+
| 5     |       |       |
| 7     |   6   |       |
| 2 4 3 |       |       |
+-------+-------+-------+
|       |       |   6   |
| 6     |       |   3   |
| 4     | 9     |   2 5 |
+-------+-------+-------+
|       |       |       |
|       |       |       |
|     4 |   5 3 | 2     |
+-------+-------+-------+



In [5]:
# Use built-in solver to obtain a solution...

puzzle.solve().show()

+-------+-------+-------+
| 5 6 9 | 3 2 8 | 1 4 7 |
| 7 1 8 | 4 6 5 | 3 9 2 |
| 2 4 3 | 1 7 9 | 8 5 6 |
+-------+-------+-------+
| 8 9 5 | 7 3 2 | 4 6 1 |
| 6 2 7 | 5 1 4 | 9 3 8 |
| 4 3 1 | 9 8 6 | 7 2 5 |
+-------+-------+-------+
| 3 5 2 | 8 9 1 | 6 7 4 |
| 9 8 6 | 2 4 7 | 5 1 3 |
| 1 7 4 | 6 5 3 | 2 8 9 |
+-------+-------+-------+



In [6]:
# Show board data encodings

puzzle.board

[[5, None, None, None, None, None, None, None, None],
 [7, None, None, None, 6, None, None, None, None],
 [2, 4, 3, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, 6, None],
 [6, None, None, None, None, None, None, 3, None],
 [4, None, None, 9, None, None, None, 2, 5],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, 4, None, 5, 3, 2, None, None]]

<h3>Let's encode the number in each cell as a one-hot encoded vector of length maxRange, 
thus, the sudoku board becomes a cube of (maxRange x maxRange x maxRange) binary variables</h3>

In [7]:
# Initialize cqm

cqm = ConstrainedQuadraticModel()

In [8]:
# Create maxRange x maxRange x maxRange i_j_k binary variables

selected = [[[Binary(f'SEL_{i}_{j}_{k}') for k in range(1, maxRange+1)] 
             for j in range(1, maxRange+1)] for i in range(1, maxRange+1)]

<h3>We need to add three families of constraints to solve a sudoku: <br><br>

        1. Numbers present in challenge must be preserved
        2. Numbers cannot repeat accross columns and rows
        3. Numbers cannot repeat across (size x size) cells

</h3>

In [9]:
# Create existing-numbers-in-board constraints

for i in range(0,maxRange):
    for j in range(0,maxRange):
        if puzzle.board[i][j]:
            sk = puzzle.board[i][j]-1
            for k in range(0,maxRange):
                if k == sk:
                    cqm.add_constraint(selected[i][j][k] == 1, 
                        label = f'existing_number_{i+1}_{j+1}_{k+1}')
                else:
                    cqm.add_constraint(selected[i][j][k] == 0, 
                        label = f'existing_number_{i+1}_{j+1}_{k+1}')

In [10]:
# Create axis just-one constraints

for i in range(0,maxRange):
    for j in range(0,maxRange):
        sum_x = []
        sum_y = []
        sum_z = []
        for k in range(0,maxRange):
            sum_x.append(selected[i][j][k])
            sum_y.append(selected[j][k][i])
            sum_z.append(selected[k][i][j])
        cqm.add_constraint(sum(sum_x) == 1, 
            label = f'only_one_axis_X_{i+1}_{j+1}')
        cqm.add_constraint(sum(sum_y) == 1, 
            label = f'only_one_axis_Y_{i+1}_{j+1}')
        cqm.add_constraint(sum(sum_z) == 1, 
            label = f'only_one_axis_Z_{i+1}_{j+1}')


In [11]:
# Create cell just-one constraints

for i_cell in range(0,size):
    for j_cell in range(0,size):
        for k in range(0,maxRange):
            sum_cell = []
            for i in range(0, size):
                for j in range(0, size):   
                    sum_cell.append(selected[i_cell*size+i][j_cell*size+j][k])
            cqm.add_constraint(sum(sum_cell) == 1, label = f'only_one_cell_Z_{i_cell}_{j_cell}_{k}')

In [12]:
# Check number of variables

len(cqm.variables)

729

In [13]:
# Sample solutions

sampleset = sampler.sample_cqm(cqm,time_limit=25,label="CQM Sudoku") 
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
n_solutions = len(feasible_sampleset)
if n_solutions == 0:
    print("No solutions found, try increasing time_limit!")
    assert 0==1
else:
    print("These many feasible solutions found: " + str(len(feasible_sampleset)))

These many feasible solutions found: 25


In [14]:
# Select best sample and build solution dictionary

best = feasible_sampleset.first
best.sample.items()
output = {}
for key, value in best.sample.items():
    output[key]=value

In [15]:
# Draw the solution board

draw_board(output)

+-------+-------+-------+
| 5 1 6 | 2 8 4 | 9 7 3 |
| 7 8 9 | 3 6 5 | 4 1 2 |
| 2 4 3 | 1 9 7 | 6 5 8 |
+-------+-------+-------+
| 3 9 1 | 5 2 8 | 7 6 4 |
| 6 2 5 | 7 4 1 | 8 3 9 |
| 4 7 8 | 9 3 6 | 1 2 5 |
+-------+-------+-------+
| 9 3 7 | 4 1 2 | 5 8 6 |
| 8 5 2 | 6 7 9 | 3 4 1 |
| 1 6 4 | 8 5 3 | 2 9 7 |
+-------+-------+-------+


In [16]:
# Compare with input

puzzle.show()

+-------+-------+-------+
| 5     |       |       |
| 7     |   6   |       |
| 2 4 3 |       |       |
+-------+-------+-------+
|       |       |   6   |
| 6     |       |   3   |
| 4     | 9     |   2 5 |
+-------+-------+-------+
|       |       |       |
|       |       |       |
|     4 |   5 3 | 2     |
+-------+-------+-------+

